# 신경망 학습  

* 학습  
    훈련 데이터로부터 가중치 매개변수의 최적값을 자동으로 획득하는 것을 뜻합니다.  
* 손실 함수
    신경망이 학습할 수 있도록 해주는 지표  
    손실 함수의 결괏값을 가장 작게 만드는 가중치 매개변수를 찾는 것이 목표  

## 4.1 데이터에서 학습한다!  

* 신경망의 특징: 데이터를 보고 학습, 가중치 매개변수의 값을 데이터를 보고 자동으로 결정  

### 4.1.1 데이터 주도 학습  

* 기계학습의 핵심: 데이터, 사람의 개입을 최소화 수집한 데이터로부터 패턴 발견  
    Ex)'5'를 분류하는 프로그램
    * 알고리즘을 밑바닥부터 설계하는 대신, 주어진 데이터를 잘 활용  
    신경망: 이미지에 포함된 중요한 특징까지 있는 그대로 학습.(종단간 기계학습)  
    
### 4.1.2 훈련데이터와 시험데이터  

* 훈련데이터&시험데이터  
    * 훈련데이터: 훈련데이터만을 이용하여 최적의 매개변수를 찾는다.
    * 시험데이터: 시험데이터를 사용하여 앞서 훈련한 모델의 실력을 평가, 범용능력을 평가  
    * 오버피팅: 한 데이터셋에만 지나치게 최적화된 상태
    
## 4.2 손실 함수  

* 신경망 학습에서는 현재의 상태를 '하나의 지표'(손실 함수)로 표현  
    그 지표(손실 함수)를 가장 좋게 만들어주는 가중치 매개변수의 값을 탐색  
    일반적으로 편균 제곱 오차와 교차 엔트로피 오차 사용  
    
### 4.2.1 평균 제곱 오차  
$$E = {\sum_{}^{} ({y_k-t_k})^2\over2}$$  
* y: 신경망이 추정한 값
* t: 정답 레이블
* k: 데이터 차원 수

In [3]:
import numpy as np
def mean_squared_error(y,t):
    return 0.5*np.sum((y-t)**2)

t=[0,0,1,0,0,0,0,0,0,0] #정답은 2
y=[0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0] #'2'일 확률이 가장 높다고 추정 (소프트맥스 함수의 출력)
mean_squared_error(np.array(y), np.array(t)) #아래의 예시보다 오차가 더 작다, 정답에 더 가까울 것으로 판단

0.09750000000000003

In [4]:
y=[0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0] #'7'일 확률이 가장 높다고 추정 
mean_squared_error(np.array(z), np.array(t))

0.5975

### 4.2.2 교차 엔트로피 오차  
$$E = {-\sum_{}^{} t_k ln y_k}$$  
* y:신경망의 출력  
* t:정답 레이블  
    t: 정답에 해당하는 인덱스의 원소만 1이고 나머지는 0 (원-핫 인코딩)

In [5]:
def cross_entropy_error(y,t):
    delta=1e-7 #마이너스 무한대 방지
    return -np.sum(t*np.log(y+delta))

t=[0,0,1,0,0,0,0,0,0,0]
y=[0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
cross_entropy_error(np.array(y), np.array(t))

0.510825457099338

In [6]:
y=[0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
cross_entropy_error(np.array(y), np.array(t))

2.302584092994546

### 4.2.3 미니배치 학습  
* 훈련 데이터 모두에 대한 손실함수의 합
$$E = {-{1 \over N} \times \sum_{n}^{} \sum_{k}^{} t_{nk} ln y_{nk}}$$  

* 많은 데이터를 대상으로 일일이 손실 함수를 계산하는 것은 현실적이지 않다.  
    데이터를 일부 추려 전체의 '근사치'로 이용 → 미니배치 학습

In [7]:
#mnist dataset을 읽어오는 코드
import sys, os 
os.chdir("C:\\Users\\신현승\\deeplearning\\dataset")
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist
(x_train, t_train), (x_test, t_test)=\
    load_mnist(normalize=True, one_hot_label=True)

print(x_train.shape) #훈련 데이터는 60000개, 입력 데이터는 784열
print(t_train.shape) #정답 레이블은 10줄 따리 데이터

(60000, 784)
(60000, 10)


In [8]:
#무작위로 10장만 빼내는 코드
train_size=x_train.shape[0]
batch_size=10
batch_mask=np.random.choice(train_size, batch_size)
x_batch=x_train[batch_mask]
t_batch=t_train[batch_mask]

np.random.choice(60000,10)

array([14101, 32250, 19699,  5687, 13309, 18884, 46450, 53078,  3821,
       49494])

### 4.2.4 (배치용) 교차 엔트로피 오차 구현하기  

In [9]:
def cross_entropy_error(y,t):
    if y.ndim==1:
        t=t.reshape(1, t.size)
        y=y.reshape(1, y.size)
        
    bath_size=y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size),t]+1e-7))/batch_size

* y: 신경망 출력  
    y가 1차원이라면(데이터 하나당 교차 엔트로피 오차를 구하는 경우) reshape 함수로 데이터 형상을 바꿔줍니다.  
    배치의 크기로 나눠 정규화하고 이미지 1장당 평균의 교차 엔트로피 오차를 계산합니다.
* t: 정답 레이블

### 4.2.5 왜 손실 함수를 설정하는가?  

* 궁극적인 목표: 높은 '정확도'를 끌어내는 매개변수 값을 찾는 것.  
* 신경망 학습에서의 '미분'의 역할에 주목  
    최적의 매개변수를 탐색 → 손실 함수의 값을 가능한 작게하는 매개변수 값을 찾음  
    →매개병수의 미분 값을 단서로 매개변수의 값을 갱신하는 과정 반복
* 미분 값이 음수이면 가중치 매개변수를 양의 방향으로 변화시켜 손실 함수의 값을 줄임  
    미분 값이 양수이면 가중치 매개변수를 음의 방향으로 변화시켜 손실 함수의 값을 줄임  
* 미분 값이 0 이면 가중치 매개변수의 갱신 멈춤 → '정확도'가 아닌 '손실 함수의 값'을 지표로 삼는 이유  
    Ex1) 100장의 훈련 데이터 중 32장을 올바로 인식  
    이때 정확도는 32%이지만 매개변수를 약간만 조정해서는 정확도가 개선되지 않고 개선된다 하더라도 불연속적인 값으로 바뀜  
    '계단 함수'를 활성화 함수로 사용하지 않는 이유와도 일치